In [17]:
import pandas as pd
import numpy as np
import os
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import shapefile  # Install with pip install pyshp
from cartopy.feature import ShapelyFeature
import geopandas as gpd
from adjustText import adjust_text
from shapely.geometry import Point

Datasets:

SMB in the US and Canada: GBIF.org (14 February 2025) GBIF Occurrence Download https://doi.org/10.15468/dl.hav6m2 0000402-250214102907787.zip


NRD in the US and Canada: GBIF.org (14 February 2025) GBIF Occurrence Download https://doi.org/10.15468/dl.a9mhaw 0000403-250214102907787.zip


In [1]:
!unzip ../0000402-250214102907787.zip ; unzip ../0000403-250214102907787.zip

Archive:  ../0000402-250214102907787.zip
  inflating: 0000402-250214102907787.csv  
Archive:  ../0000403-250214102907787.zip
  inflating: 0000403-250214102907787.csv  


In [70]:
!unzip ../lhy_000c16a_e.zip

Archive:  ../lhy_000c16a_e.zip
  inflating: lhy_000c16a_e.dbf       
  inflating: lhy_000c16a_e.prj       
  inflating: lhy_000c16a_e.shp       
  inflating: lhy_000c16a_e.shx       
  inflating: lakes_rivers_poly.html  
  inflating: 92-160-g2016002-eng.pdf  


In [95]:
smb = pd.read_csv("0000402-250214102907787.csv", sep = "\t")
nrd = pd.read_csv("0000403-250214102907787.csv", sep = "\t")

In [141]:
def overlapping_points(indf, columns):
    gdf1 = gpd.read_file('../geotiffs/Priority_Waterbody_List_-_Lakes.shp')
    gdf1 = gdf1.to_crs(epsg=4857)
    gdf1['area m2'] = gdf1.geometry.area
    gdf1['area km2'] = gdf1['area m2'] / 1000000
    gdf1 = gdf1.to_crs(epsg=4326)
    # Convert to GeoDataFrame
    gdf_points = gpd.GeoDataFrame(indf, 
                  geometry=gpd.points_from_xy(indf['decimalLongitude'], indf['decimalLatitude']), 
                  crs="EPSG:4326")  # Assuming WGS84 (decimal degrees)
    gdf_joined = gpd.sjoin(gdf_points, gdf1[['NAME', 'geometry', "area km2"]], how="left", predicate="within")
    df = gdf_joined[columns][gdf_joined["NAME"].notna()]
    return df

In [142]:
tmp1 = overlapping_points(smb, ["species", "year", "decimalLatitude", "decimalLongitude", "NAME", "area km2"])
tmp1 = tmp1.sort_values(["NAME", "year"]).reset_index(drop=True)
tmp1_dedup = tmp1.drop_duplicates("NAME").reset_index(drop=True)
tmp1_dedup = tmp1_dedup[~tmp1_dedup["NAME"].str.contains("River|river")].reset_index(drop=True)

In [143]:
tmp2 = overlapping_points(nrd, ["species", "year", "decimalLatitude", "decimalLongitude", "NAME", "area km2"])
tmp2 = tmp2.sort_values(["NAME", "year"]).reset_index(drop=True)
tmp2_dedup = tmp2.drop_duplicates("NAME", keep = "last").reset_index(drop=True)
tmp2_dedup = tmp2_dedup[~tmp2_dedup["NAME"].str.contains("River|river")].reset_index(drop=True)

In [144]:
tmp1_dedup.merge(tmp2_dedup, on = "NAME")

,species_x,year_x,decimalLatitude_x,decimalLongitude_x,NAME,area km2_x,species_y,year_y,decimalLatitude_y,decimalLongitude_y,area km2_y
0,Micropterus dolomieu,2020,44.147370,-74.899733,Cranberry Lake,55.455630,Chrosomus eos,1931.0,44.113394,-74.897372,55.455630
1,Micropterus dolomieu,2021,44.115022,-74.626162,Hitchins Pond,1.970008,Chrosomus eos,1933.0,44.116071,-74.646531,1.970008
2,Micropterus dolomieu,2023,43.826533,-74.338767,Rock Lake,1.741703,Chrosomus eos,1932.0,43.827780,-74.335560,1.741703


In [155]:
def overlapping_points(indf, columns):
    gdf1 = gpd.read_file('lhy_000c16a_e.shp')
    gdf1 = gdf1.to_crs(epsg=4857)
    gdf1['area m2'] = gdf1.geometry.area
    gdf1['area km2'] = gdf1['area m2'] / 1000000
    gdf1 = gdf1.to_crs(epsg=4326)
    # Convert to GeoDataFrame
    gdf_points = gpd.GeoDataFrame(indf, 
                  geometry=gpd.points_from_xy(indf['decimalLongitude'], indf['decimalLatitude']), 
                  crs="EPSG:4326")  # Assuming WGS84 (decimal degrees)
    gdf_joined = gpd.sjoin(gdf_points, gdf1[['NAME', 'geometry', "area km2", "HYDROUI"]], how="left", predicate="within")
    df = gdf_joined[columns][gdf_joined["NAME"].notna()]
    return df

In [156]:
tmp1 = overlapping_points(smb, ["species", "year", "decimalLatitude", "decimalLongitude", "NAME", "area km2", "HYDROUID"])
tmp1 = tmp1.sort_values(["HYDROUID", "year"]).reset_index(drop=True)
tmp1_dedup = tmp1.drop_duplicates("HYDROUID").reset_index(drop=True)
tmp1_dedup = tmp1_dedup[~tmp1_dedup["HYDROUID"].str.contains("River|river")].reset_index(drop=True)

KeyError: "['HYDROUID'] not in index"

In [ ]:
tmp2 = overlapping_points(nrd, ["species", "year", "decimalLatitude", "decimalLongitude", "NAME", "area km2", "HYDROUID"])
tmp2 = tmp2.sort_values(["HYDROUID", "year"]).reset_index(drop=True)
tmp2_dedup = tmp2.drop_duplicates("HYDROUID", keep = "last").reset_index(drop=True)
tmp2_dedup = tmp2_dedup[~tmp2_dedup["HYDROUID"].str.contains("River|river")].reset_index(drop=True)

In [ ]:
tmp2

In [148]:
tmp1_dedup.merge(tmp2_dedup, on = "NAME")

,species_x,year_x,decimalLatitude_x,decimalLongitude_x,NAME,area km2_x,species_y,year_y,decimalLatitude_y,decimalLongitude_y,area km2_y
0,Micropterus dolomieu,2015,44.599517,-78.796347,Balsam Lake,52.162065,Chrosomus eos,1934.0,44.654096,-78.801256,52.162065
1,Micropterus dolomieu,2017,44.777753,-76.224314,Big Rideau Lake,93.985181,Chrosomus eos,2017.0,44.775011,-76.230783,93.985181
2,Micropterus dolomieu,2020,45.252533,-76.761106,Calabogie Lake,27.507091,Chrosomus eos,1984.0,45.266700,-76.766700,27.507091
3,Micropterus dolomieu,2023,46.028572,-78.390708,Cedar Lake,50.408908,Chrosomus eos,1936.0,46.016667,-78.466667,50.408908
4,Micropterus dolomieu,2024,47.378053,-79.745863,Gillies Lake,2.823829,Chrosomus eos,1958.0,47.383300,-79.733300,2.823829
5,Micropterus dolomieu,2024,45.050640,-78.514926,Head Lake,1.262248,Chrosomus eos,1935.0,45.516667,-78.566667,2.023440
6,Micropterus dolomieu,2023,45.988966,-73.999353,Lac Cromwell,0.164188,Chrosomus eos,2023.0,45.989250,-73.998225,0.164188
7,Micropterus dolomieu,2022,45.600895,-75.997658,Lac Philippe,3.167453,Chrosomus eos,1990.0,45.603900,-76.004400,3.167453
8,Micropterus dolomieu,2023,46.205454,-75.825404,Lac des Trente et Un Milles,91.446373,Chrosomus eos,1983.0,46.200000,-75.816700,91.446373
9,Micropterus dolomieu,2019,44.519505,-79.420897,Lake Simcoe,1304.885352,Chrosomus eos,2014.0,44.474118,-79.191471,1304.885352
